### Image analysis with PlantCV Application

Analyze Teff Images

Select your images by entering your name [HERE](https://docs.google.com/spreadsheets/d/1taqJsPOu8LDW-5F5eS3GRWlV5UhHZHZGJCKr5X17_eI/edit#gid=0)

[PlantCV documentation page](https://plantcv.readthedocs.io/en/stable/)

In [ ]:
%matplotlib inline
import os
import cv2
import numpy as np
from plantcv import plantcv as pcv

In [ ]:
# Print the PlantCV version
pcv.__version__

In [ ]:
# Handle input variables
# This code was provided to make it easier to convert the notebook to a script
class options():
    def __init__(self):
        self.debug    = "plot"
        self.result   = "results.csv"
        self.outdir   = "./output_images"
        self.writeimg = True
        self.image    = "/shares/mgehan_share/mgehan/getu-tef/images/..."  
#                         ^
#                         |
#                Finish this by filling in the full file name of the image in "..."

In [ ]:
# Create input arguments object


In [ ]:
# Set debug mode and debug outdir


In [ ]:
# Create a folder to save the result image files if the folder not exists


In [ ]:
# Update params related to plotting so we can see better 


#### STEP 1: Open an image
We start with reading the image 

In [ ]:
# Read in the image using "readimage" function in plantcv
# Hint: pcv.readimage
img, imgpath, imgname = 

#### STEP 2: Find the color card and correct the exposure of the image
The color card can be find automatically or defined location manually

In [ ]:
# STEP 2: White balance
img1 = pcv.white_balance(img=img, mode='max', roi=(x, y, width, height))
#                                                  ^               ^   
#                                                  |_______________|   
#                                       
#                                adjust these these four values, most cases there is no need to change the last two valuse (50) 

#### STEP 3: Visualize colorspaces in PlantCV
Visualize the Hue, Saturation, Value (HSV; top row), Lightness, Green-Magenta, Blue-Yellow (LAB; bottom row) colorspaces. We looked for a color channel that enhanced the difference between plant and background. The workshop group decided the Blue-Yellow ("b") channel was the choice.

In [ ]:
# Visualize all colorspaces the image using the "colorspaces" function in plantcv's "visualize" subpackage
# Hint: pcv.visualize.colorspaces
all_cs = 

#### STEP 4: Report size marker
Get and record the size of a size marker or set an area as a size marker.

In [ ]:
# Define an ROI for the marker
# Hint: pcv.roi.rectangle
roi_contour, roi_hierarchy =

# Detect and Measure Size Marker
# Hint: pcv.report_size_marker_area

#### STEP 5: Convert image from RGB colorspace to LAB colorspace
Extract the Green-Magenta ("a") channel channel from the LAB colorspace.
Other availabilities from LAB colorspace:
lightness ("l")
Blue-Yellow ("b")

In [ ]:
%matplotlib notebook
# Convert from RGB to LAB colorspace and keep only channel "a" using the plantcv function "rgb2gray_lab"
# Hint: pcv.rgb2gray_lab

gray_img = 

#### STEP 6: Plot a histogram of pixel values for the selected color channel
Green-Magenta ("a") channel.

In [ ]:
# Visualize the histogram for the grayscale image using "histogram" function in the plantcv subpackage "visualize"
# Hint: pcv.visualize.histogram


#### STEP 7: Apply a binary threshold to the Green-Magenta ("a") channel grayscale image.
In the histogram (in STEP 5) the biggest peak is the uniform background in the image above. 
Through pixel inspection in "notebook" mode (in STEP 4) we saw that the smaller peak between ~110 to 116 are the plant pixels.

This gives us a hint of choosing threshold value. 
We choose a threshold value of 115 to set pixels with values > 115 to black (0) and <= 115 to white (255).

In [ ]:
%matplotlib inline
# Apply a binary threshold to the "a" channel by function "binary"
# Hint: pcv.threshold.binary

thresh_img = 

#### STEP 8: Create a region of interest (ROI)
We have removed most or all of the background around the plant but still have some large background objects (e.g. color card, pot label, etc.). An ROI will allow us to keep objects only in the region we want. Since we know in this dataset that the plants are always centered in the middel of the image we can set an ROI easily. The ROI is centered at 1725, 1155 and has a radius of 400 pixels.

In [ ]:
# Create a rectangular ROI
# Hint: pcv.roi.rectangle
roi, roi_str =               

#### STEP 9: Identify objects in the binary image
Before we can filter out background objects we have to identify objects in the binary image above. The `find_objects` function identifies contours (or connected components) and returns a polygon for each. The `cnts` and `cnts_str` are the list of polygons and the hierarchical relationship between polygons, respectively.

In [ ]:
# Identify objects in the binary image using "find_objects" function in plantcv
# Hint: pcv.find_objects
cnts, cnts_str = 

#### STEP 10: Filter objects by a region of interest (ROI)
Now we combine the ROI and polygons found in the previous two steps. The `roi_objects` function will use the ROI to find polygons that overlap and keep them. Polygons completely outside the ROI will be discarded. The polygons need not be entirely enclosed by the ROI, for flexibility, because we are using the mode "partial."

In [ ]:
# Filter objects by region of interest using "roi_objects" function
# Hint: pcv.roi_objects
plant_cnt, plant_str, plant_mask, plant_area = 

#### STEP 11: Combine remaining objects into a single object
To collect measurements we need to build a single plant object from potentially multiple objects/polygons. For example, sometimes leaves are detected as separate objects.

In [ ]:
# Combine objects into one using "object_composition" function
# Hint: pcv.object_composition
plant, mask = 

#### STEP 12: Measure plant size and shape
The `analyze_object` function outputs measurements about the size and shape of the plant. If the `writeimg` input argument is `True` we will save the output image to a file.

In [ ]:
# Measure size and shape properties
# Hint: pcv.analyze_object
shape_img = 


#### STEP 13: (Optional) Analyze color of plant
The `analyze_color` function outputs histograms of choice (indicated by `colorspaces`). If the `writeimg` input argument is `True` we will save the output image to a file.

In [ ]:
# Analyze color by examining histograms using the "analyze_color" function
# Hint: pcv.analyze_color
color_img = 

#### STEP 14: Set horizontal boundary line
This allows the extraction of biologically meaningful data.

##### STEP 14.1 (Optional) Rotate the image and the mask

In [ ]:
# Rotate the original image if necessary using the "rotate" function
# Hint: pcv.transform.rotate
rotate_img = 

In [ ]:
# rotate the mask using the EXACT same value
img  = rotate_img
mask = 

##### STEP 14.2 Set the horizontal reference line to split the measurements into "above line" and "below line"

In [ ]:
# Set the horizontal reference line using "analyze_bound_horizontal"
# Hint: pcv.analyze_bound_horizontal
boundary_image = 

#### STEP 15: Save the measurements to a file
The `save_results` function saves the results collected by all the `analyze_*` functions.

In [ ]:
# Save the measurements to a file
# Hint: pcv.outputs.save_results with outformat="csv"


#### STEP 16: Try look at some measurements you've collected!
Feel free to take a look at some measurements you've collected, I've included some examples below.

Also take a look to see if your result file is saved!